In [18]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
%matplotlib inline 

In [19]:

telco_raw = pd.read_csv("C:/Users/pdamodh1/Downloads/attrition_telco.csv",skipinitialspace=True)
print(len(telco_raw.index))
print(len(telco_raw.columns))
telco_raw.head()

100000
173


,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,rev_Range,...,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,car_buy,retdays,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,25.99,...,U,U,U,U,U,Y,New,23.0,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,153.14,...,U,U,U,U,U,Y,UNKNOWN,NaN,240.0,1000002
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,0.00,...,U,Y,U,U,U,Y,New,NaN,1504.0,1000003
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,0.00,...,Y,U,U,U,U,Y,New,NaN,1812.0,1000004
4,55.2300,570.50,71.980,0.0000,0.00,0.0,0.0,0.0,0.0,0.00,...,U,U,U,U,U,Y,New,NaN,434.0,1000005


In [20]:
#telco datatypes 
telco_data_types =list(zip(list(telco_raw),telco_raw.dtypes))
tl_dtype_frame=pd.DataFrame(data =telco_data_types, columns=['colname', 'dtype'])
tl_dtype_frame.to_csv("C:/Users/pdamodh1/Downloads/dtype.csv", sep=',')
tl_dtype_frame

,colname,dtype
0,rev_Mean,float64
1,mou_Mean,float64
2,totmrc_Mean,float64
3,da_Mean,float64
4,ovrmou_Mean,float64
5,ovrrev_Mean,float64
6,vceovr_Mean,float64
7,datovr_Mean,float64
8,roam_Mean,float64
9,rev_Range,float64


In [21]:
# basic summary stat and storing to csv . Note : storing to csv because in pandas browser we cannot view th whole dataframe 
desc=telco_raw.describe().T
desc.to_csv("C:/Users/pdamodh1/Downloads/decs.csv", sep=',')
desc

C:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,count,mean,std,min,25%,50%,75%,max
rev_Mean,99643.0,5.871998e+01,46.291677,-6.167500e+00,NaN,NaN,NaN,3.843262e+03
mou_Mean,99643.0,5.135599e+02,525.168140,0.000000e+00,NaN,NaN,NaN,1.220675e+04
totmrc_Mean,99643.0,4.617914e+01,23.623489,-2.691500e+01,NaN,NaN,NaN,4.099900e+02
da_Mean,99643.0,8.888279e-01,2.177619,0.000000e+00,NaN,NaN,NaN,1.593900e+02
ovrmou_Mean,99643.0,4.107225e+01,97.296150,0.000000e+00,NaN,NaN,NaN,4.320750e+03
ovrrev_Mean,99643.0,1.355956e+01,30.500885,0.000000e+00,NaN,NaN,NaN,1.102400e+03
vceovr_Mean,99643.0,1.329506e+01,30.056089,0.000000e+00,NaN,NaN,NaN,8.960875e+02
datovr_Mean,99643.0,2.613177e-01,3.126531,0.000000e+00,NaN,NaN,NaN,4.235400e+02
roam_Mean,99643.0,1.286405e+00,14.711374,0.000000e+00,NaN,NaN,NaN,3.685200e+03
rev_Range,99643.0,4.462412e+01,88.135464,0.000000e+00,NaN,NaN,NaN,1.374054e+04


# Data cleansing step

In [22]:
#undesrtanding the missing data varibale 
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns 
missing_values_table(telco_raw).to_csv("C:/Users/pdamodh1/Downloads/Mis_data_percentage_2.csv", sep=',')
x=missing_values_table(telco_raw)
x

,Missing Values,% of Total Values
rev_Mean,357,0.357
mou_Mean,357,0.357
totmrc_Mean,357,0.357
da_Mean,357,0.357
ovrmou_Mean,357,0.357
ovrrev_Mean,357,0.357
vceovr_Mean,357,0.357
datovr_Mean,357,0.357
roam_Mean,357,0.357
rev_Range,357,0.357


In [23]:
#For the removed the columns that are 30 % missing ,
# Iam  sure you have quetions why ? ,We can impuet the column but the data missing more than 30 % will be baised in our analysis
#even the less data our model will be stable 
x2=list(x.index[x['% of Total Values']<30])
telco_col_removed=telco_raw[x2]

In [25]:
# Impuataion by LOCF Method and Mean based method for the imputataion 
# here i have used the standard mean based imputaion and 
# column vise data Analyis and Impuataion
# Here forthe Column named "Mean" I have applied the Overall mean of the column 
Mean_Df=tl_dtype_frame[tl_dtype_frame['colname'].str.contains("Mean")]
Meanlist=Mean_Df['colname']
def Impute(x,df):
    df[x].fillna(df[x].mean(), inplace=True)
    return 
#for i in Meanlist:
#Impute(i,telco_col_removed)
for i in Meanlist:
    Impute(i,telco_col_removed)

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


# I Have mean based impuataion for other columns based 
# reasons 1.Business impact we dont know
# 2.Interpolation will not work beacause its not a seqential data
This affect not on the distribution of the data

In [26]:
Mean_Df=tl_dtype_frame[tl_dtype_frame['colname'].str.contains("Range")]
Meanlist=Mean_Df['colname']
def Impute(x,df):
    df[x].fillna(df[x].mean(), inplace=True)
    return 
#for i in Meanlist:
#Impute(i,telco_col_removed)
for i in Meanlist:
    Impute(i,telco_col_removed)

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [27]:
# saving the missing callculation to file 
# checking the missing percentgae again 
missing_values_table(telco_col_removed).to_csv("C:/Users/pdamodh1/Downloads/after_impute1.csv", sep=',')
missing_values_table(telco_col_removed)

,Missing Values,% of Total Values
rev_Mean,0,0.000
mou_Mean,0,0.000
totmrc_Mean,0,0.000
da_Mean,0,0.000
ovrmou_Mean,0,0.000
ovrrev_Mean,0,0.000
vceovr_Mean,0,0.000
datovr_Mean,0,0.000
roam_Mean,0,0.000
rev_Range,0,0.000


# Removing My row which still have nulls 

In [28]:
# saving a copy of df for future use on call
telco_col_removed1=telco_col_removed

# Removing the rows 
telco_col_removed=telco_col_removed.dropna( how='any')
missing_values_table(telco_col_removed)
telco_col_removed.describe().T
print(len(telco_col_removed.index))
print(len(telco_col_removed.columns))
missing_values_table(telco_col_removed).to_csv("C:/Users/pdamodh1/Downloads/after_impute_fin1.csv", sep=',')

60385
145


In [29]:
#undestanding columns after columns removed
telco_data_types1 =list(zip(list(telco_col_removed),telco_col_removed.dtypes))
tl_dtype_frame1=pd.DataFrame(data=telco_data_types1, columns=['colname', 'dtype'])
#df_total slection=tl_dtype_frame1.loc[tl_dtype_frame1['dtype'] == int64]
#telco_col_rm
#obj_columns=df_objectslection['colname']
counts = tl_dtype_frame1.groupby('dtype').size()
print(counts)

#----------------------------------------------
# slecting the object columns and understiang it for recode 

df_objectslection1=tl_dtype_frame1.loc[tl_dtype_frame1['dtype'] == object]
obj_columns=df_objectslection1['colname']
Obj_df = pd.DataFrame(telco_col_removed, columns=obj_columns)

for column in Obj_df:
    print(column)
    print(list(Obj_df[column].unique()))

dtype
int64      37
float64    89
object     19
dtype: int64
new_cell
['U', 'Y', 'N']
crclscod
['A', 'C', 'BA', 'B', 'D', 'G', 'CA', 'AA', 'E', 'EA', 'U', 'J', 'W', 'Y', 'Z', 'CC', 'GA', 'DA', 'ZA', 'JF', 'K', 'Z1', 'O', 'U1', 'E4', 'A2', 'V', 'M', 'I', 'TP', 'C5', 'Z4', 'C2', 'EM', 'D5', 'Z5', 'D4', 'GY', 'E2', 'A3', 'V1', 'CY', 'B2', 'EC', 'D2', 'Z2', 'ZY', 'P1', 'H', 'EF', 'L']
asl_flag
['N', 'Y']
prizm_social_one
['S', 'U', 'C', 'T', 'R']
csa
['SEAPOR503', 'NNYSYR315', 'PITHOM412', 'OKCTUL918', 'OKCOKC405', 'SANMCA210', 'SLCSLC801', 'MILMIL414', 'LOULOU502', 'KCYKCK913', 'KCYNEW316', 'KCYKCM816', 'DENDEN303', 'PHICTR610', 'OKCLRK501', 'SLCPRO801', 'SLCOGD801', 'OMAIWC319', 'SANAUS512', 'DALFTW817', 'PHXTUC520', 'OMALNC402', 'NSHNSH615', 'OMAOMA402', 'INDIND317', 'SANSAN210', 'NYCMAN917', 'NOLKEN504', 'DALSHR903', 'SEASPO509', 'MINMIN612', 'DALDAL214', 'NYCQUE917', 'SANGEO512', 'OMADES515', 'STLSTL314', 'PHXPHX602', 'BOSPTL207', 'INDMUN765', 'MIAMIA305', 'NNYBUF716', 'PHIPHI215', 'P

In [30]:
to_drop = ['crclscod','csa','area','ethnic']
telco_col_removed.drop(to_drop, axis=1, inplace=True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [31]:
telco_col_removed['asl_flag'] = telco_col_removed['asl_flag'].map({'Y': 1, 'N': 0})
telco_col_removed['creditcd'] = telco_col_removed['creditcd'].map({'Y': 1, 'N': 0})
telco_col_removed['kid16_17'] = telco_col_removed['kid16_17'].map({'U':1, 'Y': 0})
telco_col_removed['kid11_15'] = telco_col_removed['kid11_15'].map({'U': 1, 'Y': 0})
telco_col_removed['kid6_10'] = telco_col_removed['kid6_10'].map({'U': 1, 'Y': 0})
telco_col_removed['kid3_5'] = telco_col_removed['kid3_5'].map({'U': 1, 'Y': 0})
telco_col_removed['kid0_2'] = telco_col_removed['kid0_2'].map({'U': 1, 'Y': 0})
telco_col_removed['car_buy'] = telco_col_removed['car_buy'].map({'New': 1, 'UNKNOWN': 0})
telco_col_removed['hnd_webcap'] = telco_col_removed['hnd_webcap'].map({'WCMB': 1, 'WC': 0})
telco_col_removed['refurb_new'] = telco_col_removed['refurb_new'].map({'N': 1, 'R': 0})
#telco_col_removed['new_cell'] = telco_col_removed['newcell'].map({'Y': 1, 'N': 0,'U':0})


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [32]:
#again running for seeing the columns that have changed 
telco_data_types1 =list(zip(list(telco_col_removed),telco_col_removed.dtypes))
tl_dtype_frame1=pd.DataFrame(data=telco_data_types1, columns=['colname', 'dtype'])
#df_total slection=tl_dtype_frame1.loc[tl_dtype_frame1['dtype'] == int64]
#telco_col_rm
#obj_columns=df_objectslection['colname']
counts = tl_dtype_frame1.groupby('dtype').size()
print(counts)

#----------------------------------------------
# slecting the object columns and understiang it for recode 

df_objectslection1=tl_dtype_frame1.loc[tl_dtype_frame1['dtype'] == object]
obj_columns=df_objectslection1['colname']
Obj_df = pd.DataFrame(telco_col_removed, columns=obj_columns)

for column in Obj_df:
    print(column)
    print(list(Obj_df[column].unique()))

dtype
int64      47
float64    89
object      5
dtype: int64
new_cell
['U', 'Y', 'N']
prizm_social_one
['S', 'U', 'C', 'T', 'R']
dualband
['Y', 'N', 'T']
marital
['S', 'M', 'A', 'U', 'B']
infobase
['M']


In [116]:

to_drop = ['crclscod','csa','area','ethnic']
telco_col_removed.drop(to_drop, axis=1, inplace=True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [ ]:
#removing crclscod,,csa,area beacosue of the model logistic regreesion requires only binary and integrer not the categoricalbase
#--------------------------------------------------------------
# we are going to change the new_cell column to binary 
# we are going to change the asl_flag column to binary 
#new_cell,asl_flag,refurb_new

#-----------------------------------------------------------------------
#dummy variables or the ( tranforming the variableto binary form through the dummy variable )



In [39]:
dummy_new_cell = pd.get_dummies(telco_col_removed['new_cell'], prefix='new_cell')
telco_col_removed3=telco_col_removed
#telco_col_removed.drop('new_cell')
newcell_added = pd.concat([telco_col_removed,dummy_new_cell],axis=1)
#list(newcell_added)
#newcell_added.drop('new_cell',axis=1)
newcell_added.drop('new_cell',axis=1,inplace=True)
#list(newcell_added)
#prizm_social_one dummy addding 
dummy_prizm_social_one = pd.get_dummies(newcell_added['prizm_social_one'], prefix='prizm_social_one')
#dummy_prizm_social_one
prizm_added = pd.concat([newcell_added,dummy_prizm_social_one],axis=1)
prizm_added.drop('prizm_social_one',axis=1,inplace=True)
#list(prizm_added)
#dual band  dummy added
dummy_dualband = pd.get_dummies(prizm_added['dualband'], prefix='dualband')
dualband_added = pd.concat([prizm_added,dummy_dualband],axis=1)
dualband_added.drop('dualband',axis=1,inplace=True)
#list(dualband_added)
# marital
dummy_marital = pd.get_dummies(dualband_added['marital'], prefix='marital')
marital_added = pd.concat([dualband_added,dummy_marital],axis=1)
marital_added.drop('marital',axis=1,inplace=True)
list(marital_added)


['rev_Mean',
 'mou_Mean',
 'totmrc_Mean',
 'da_Mean',
 'ovrmou_Mean',
 'ovrrev_Mean',
 'vceovr_Mean',
 'datovr_Mean',
 'roam_Mean',
 'rev_Range',
 'mou_Range',
 'totmrc_Range',
 'da_Range',
 'ovrmou_Range',
 'ovrrev_Range',
 'vceovr_Range',
 'datovr_Range',
 'roam_Range',
 'change_mou',
 'change_rev',
 'drop_vce_Mean',
 'drop_dat_Mean',
 'blck_vce_Mean',
 'blck_dat_Mean',
 'unan_vce_Mean',
 'unan_dat_Mean',
 'plcd_vce_Mean',
 'plcd_dat_Mean',
 'recv_vce_Mean',
 'recv_sms_Mean',
 'comp_vce_Mean',
 'comp_dat_Mean',
 'custcare_Mean',
 'ccrndmou_Mean',
 'cc_mou_Mean',
 'inonemin_Mean',
 'threeway_Mean',
 'mou_cvce_Mean',
 'mou_cdat_Mean',
 'mou_rvce_Mean',
 'owylis_vce_Mean',
 'mouowylisv_Mean',
 'iwylis_vce_Mean',
 'mouiwylisv_Mean',
 'peak_vce_Mean',
 'peak_dat_Mean',
 'mou_peav_Mean',
 'mou_pead_Mean',
 'opk_vce_Mean',
 'opk_dat_Mean',
 'mou_opkv_Mean',
 'mou_opkd_Mean',
 'drop_blk_Mean',
 'attempt_Mean',
 'complete_Mean',
 'callfwdv_Mean',
 'callwait_Mean',
 'drop_vce_Range',
 'drop_da

In [40]:
#checking the data
telco_data_marital_added =list(zip(list(marital_added),marital_added.dtypes))
tl_dtype_marital_added=pd.DataFrame(data=telco_data_marital_added, columns=['colname', 'dtype'])
#df_total slection=tl_dtype_frame1.loc[tl_dtype_frame1['dtype'] == int64]
#telco_col_rm
#obj_columns=df_objectslection['colname']
counts = tl_dtype_marital_added.groupby('dtype').size()
print(counts)

dtype
int64       47
float64    105
object       1
dtype: int64


In [46]:
#now the drop the infobasecolumns  is scalar we can start with the modelinnn
marital_added.drop('infobase',axis=1,inplace=True)

In [37]:
missing_values_table(telco_col_removed).to_csv("C:/Users/pdamodh1/Downloads/missing_final1.csv", sep=',')

In [47]:
marital_added.to_csv("C:/Users/pdamodh1/Downloads/final_data2.csv", sep=',')

# Correlation Analysis to understand the corleation of the model
Here we done the correlation analysis to avoid the highly corelated variables this helps the model to stablise and avoid the over fitting
Note : we can also try the PCA method whitout doing this , but by doing that we may lose the identity of the variable


In [48]:
Finaldf=marital_added
cormat=Finaldf.corr()
cormat.to_csv("C:/Users/pdamodh1/Downloads/correlation_matrix.csv", sep=',')

# Stat funtion for the Logistic regression

In [49]:
train_cols=Finaldf.ix[:, Finaldf.columns !='churn']
#Simple Logit model for thr curn
import statsmodels.api as sm
#newdf1=newdf.loc[:, newdf.dtypes == np.float64]
#train_cols=newdf1.ix[:, newdf1.columns !='churn']
#train_cols = telco_col_removed1[-]
  # Index([gre, gpa, prestige_2, prestige_3, prestige_4], dtype=object)
logit = sm.Logit(Finaldf["churn"],train_cols)
result = logit.fit()

         Current function value: 0.666283
         Iterations: 35


C:\Program Files\Anaconda3\lib\site-packages\statsmodels\base\model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [50]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  churn   No. Observations:                60385
Model:                          Logit   Df Residuals:                    60241
Method:                           MLE   Df Model:                          143
Date:                Tue, 31 Jan 2017   Pseudo R-squ.:                 0.03728
Time:                        16:27:53   Log-Likelihood:                -40234.
converged:                      False   LL-Null:                       -41791.
                                        LLR p-value:                     0.000
======================================================================================
                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
rev_Mean               0.0087      0.003      2.522      0.012         0.002     0.016
mou_Mean              -0.0008      0.000     -1.778      0.075        -0.002  8.57e-05
totmrc_Mean           -0.0031      0.001     -3.072      0.002        -0.005    -0.001
da_Mean                0.0042      0.008      0.521      0.603        -0.012     0.020
ovrmou_Mean            0.0003      0.001      0.608      0.543        -0.001     0.001
ovrrev_Mean           -0.1025      0.081     -1.262      0.207        -0.262     0.057
vceovr_Mean            0.1043      0.081      1.284      0.199        -0.055     0.263
datovr_Mean            0.1065      0.082      1.295      0.195        -0.055     0.268
roam_Mean              0.0129      0.004      3.650      0.000         0.006     0.020
rev_Range              0.0007      0.000      1.606      0.108        -0.000     0.002
mou_Range              0.0004   4.33e-05      8.889      0.000         0.000     0.000
totmrc_Range          -0.0041      0.001     -7.666      0.000        -0.005    -0.003
da_Range               0.0047      0.006      0.835      0.404        -0.006     0.016
ovrmou_Range           0.0003      0.000      0.865      0.387        -0.000     0.001
ovrrev_Range          -0.0041      0.004     -1.054      0.292        -0.012     0.004
vceovr_Range           0.0040      0.004      1.056      0.291        -0.003     0.011
datovr_Range           0.0005      0.005      0.104      0.918        -0.010     0.011
roam_Range            -0.0037      0.001     -3.234      0.001        -0.006    -0.001
change_mou            -0.0005      0.000     -2.911      0.004        -0.001    -0.000
change_rev            -0.0004      0.001     -0.336      0.737        -0.003     0.002
drop_vce_Mean        -28.8535    2.4e+04     -0.001      0.999      -4.7e+04   4.7e+04
drop_dat_Mean       1233.4058   5.57e+04      0.022      0.982     -1.08e+05   1.1e+05
blck_vce_Mean        -28.8605    2.4e+04     -0.001      0.999      -4.7e+04   4.7e+04
blck_dat_Mean       1233.4434   5.57e+04      0.022      0.982     -1.08e+05   1.1e+05
unan_vce_Mean         -0.0099      0.007     -1.458      0.145        -0.023     0.003
unan_dat_Mean       1262.2272        nan        nan        nan           nan       nan
plcd_vce_Mean        -10.8503   1433.793     -0.008      0.994     -2821.033  2799.333
plcd_dat_Mean      -1273.1390        nan        nan        nan           nan       nan
recv_vce_Mean          0.0007      0.001      0.516      0.606        -0.002     0.003
recv_sms_Mean          0.0043      0.015      0.280      0.779        -0.026     0.035
comp_vce_Mean         11.4668        nan        nan        nan           nan       nan
comp_dat_Mean       1301.6210        nan        nan        nan           nan       nan
custcare_Mean          0.0393      0.011      3.475      0.001         0.017     0.062
ccrndmou_Mean         -0.0734      0.022     -3.385      0.001        -0.116    -0.031
cc_mou_Mean           

In [51]:
print(result.conf_int())

                               0             1
rev_Mean            1.945639e-03  1.552009e-02
mou_Mean           -1.758652e-03  8.569112e-05
totmrc_Mean        -5.058246e-03 -1.117677e-03
da_Mean            -1.154947e-02  1.990526e-02
ovrmou_Mean        -7.288948e-04  1.384569e-03
ovrrev_Mean        -2.616350e-01  5.671923e-02
vceovr_Mean        -5.490340e-02  2.634078e-01
datovr_Mean        -5.468636e-02  2.676778e-01
roam_Mean           5.981235e-03  1.985065e-02
rev_Range          -1.573011e-04  1.583825e-03
mou_Range           2.997828e-04  4.693773e-04
totmrc_Range       -5.099628e-03 -3.022841e-03
da_Range           -6.384671e-03  1.585413e-02
ovrmou_Range       -3.285335e-04  8.475424e-04
ovrrev_Range       -1.168925e-02  3.515874e-03
vceovr_Range       -3.424125e-03  1.141917e-02
datovr_Range       -9.570856e-03  1.063841e-02
roam_Range         -5.924295e-03 -1.452796e-03
change_mou         -7.960977e-04 -1.554086e-04
change_rev         -2.674039e-03  1.891983e-03
drop_vce_Mean

In [61]:
Final_2=Finaldf

In [63]:
Final_2=Final_2.dropna( how='any')
missing_values_table(Final_2)
print(len(Final_2.index))
print(len(Final_2.columns))


60385
152


In [68]:
missing_values_table(Final_2).to_csv("C:/Users/pdamodh1/Downloads/Final_2_missing.csv", sep=',')

In [71]:

y=Final_2['churn']
X=Final_2.drop('churn',axis=1).astype(np.float)

In [80]:
model.score(X, y)

0.59837708040076176